In [ ]:
#%pip install "autogen-ext[magentic-one,openai]" 
#%pip install gradio
# If using the MultimodalWebSurfer, you also need to install playwright dependencies:


## Magentic One Overview

Magentic-One is a generalist multi-agent system for solving open-ended web and file-based tasks across a variety of domains. It represents a significant step forward for multi-agent systems, achieving competitive performance on a number of agentic benchmarks (see the technical report for full details).

When originally released in November 2024 Magentic-One was implemented directly on the autogen-core library. We have now ported Magentic-One to use autogen-agentchat, providing a more modular and easier to use interface.

Using Magentic-One involves interacting with a digital world designed for humans, which carries inherent risks. To minimize these risks, consider the following precautions:

- Use Containers: Run all tasks in docker containers to isolate the agents and prevent direct system attacks.

- Virtual Environment: Use a virtual environment to run the agents and prevent them from accessing sensitive data.

- Monitor Logs: Closely monitor logs during and after execution to detect and mitigate risky behavior.

- Human Oversight: Run the examples with a human in the loop to supervise the agents and prevent unintended consequences.

- Limit Access: Restrict the agents’ access to the internet and other resources to prevent unauthorized actions.

- Safeguard Data: Ensure that the agents do not have access to sensitive data or resources that could be compromised. Do not share sensitive information with the agents. Be aware that agents may occasionally attempt risky actions, such as recruiting humans for help or accepting cookie agreements without human involvement. Always ensure agents are monitored and operate within a controlled environment to prevent unintended consequences. Moreover, be cautious that Magentic-One may be susceptible to prompt injection attacks from webpages.

## Example of using AI Assistant Agents & Magentic One Agents with the MAGENTIC ONE Orchestrator

## Import Required Dependencies

In [1]:
import asyncio
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
from autogen_ext.agents.file_surfer import FileSurfer
from autogen_ext.agents.magentic_one import MagenticOneCoderAgent
from autogen_agentchat.agents import CodeExecutorAgent
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from autogen_ext.teams.magentic_one import MagenticOne


## Create a Model Client with a reasoning model

In [2]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="o1",
    azure_endpoint="https://aq-aoai-swedencentral.openai.azure.com/",
    model="o1",
    api_version="2025-03-01-preview",
    api_key="3ShjYyRHd6t0VthIuoOUe1GEnwlj1tkNSHkOs6IrO1CxKQ3gHmsIJQQJ99BAACfhMk5XJ3w3AAABACOG3ddZ",
 #   temperature=0.7,
)


## Create Basic Team with one Assistant only

In [3]:
async def main() -> None:
    assistant = AssistantAgent(
        "Assistant",
        model_client=model_client,
    )
    team = MagenticOneGroupChat([assistant], model_client=model_client)
    await Console(team.run_stream(task="Provide a different proof for Fermat's Last Theorem"))
    await model_client.close()


await main()


---------- user ----------
Provide a different proof for Fermat's Last Theorem
---------- MagenticOneOrchestrator ----------

We are working to address the following user request:

Provide a different proof for Fermat's Last Theorem


To answer this request we have assembled the following team:

Assistant: An agent that provides assistance with ability to use tools.


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS  
• No specific facts or numerical data are explicitly provided in the request itself.  
 
2. FACTS TO LOOK UP  
• Historical and technical details regarding Andrew Wiles’s original proof and related advancements in number theory could be consulted from peer-reviewed mathematical journals or authoritative texts (e.g., Annals of Mathematics).  

3. FACTS TO DERIVE  
• Any intermediate results or corollaries from known theorems in algebraic geometry, number theory, or related fields that might be applied in constructing a novel proof.  

4. EDUCATED GUES

Error processing publish message for Assistant_df6b17d6-54ed-4924-b9e1-d91595bc6cb9/df6b17d6-54ed-4924-b9e1-d91595bc6cb9
Traceback (most recent call last):
  File "/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 510, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
    ...<2 lines>...
    )
    ^
  File "/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_core/_base_agent.py", line 113, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site

BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': True, 'detected': True}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}

## Create a team with a multimodal websurfer

- WebSurfer: A helpful assistant with access to a web browser. Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, filling in form fields, etc.). It can also summarize the entire page, or answer questions based on the content of the page. It can also be asked to sleep and wait for pages to load, in cases where the page seems not yet fully loaded.

In [ ]:
# The example code may download files from the internet, execute code, and interact with web pages. 
# Ensure you are in a safe environment before running the example code.
async def main() -> None:
    surfer = MultimodalWebSurfer(
        "WebSurfer",
        model_client=model_client,
    )

    team = MagenticOneGroupChat([surfer], model_client=model_client)
    await Console(team.run_stream(task="What is the UV index in Toronto today?"))




await main()


## Create a multi-agent team with the following capabilities

- WebSurfer: A helpful assistant with access to a web browser. Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, filling in form fields, etc.). It can also summarize the entire page, or answer questions based on the content of the page. It can also be asked to sleep and wait for pages to load, in cases where the page seems not yet fully loaded.

- FileSurfer: An agent that can handle local files.

- Coder: A helpful and general-purpose AI assistant that has strong language skills, Python skills, and Linux command line skills.

- ComputerTerminal: A computer terminal that performs no other action than running Python scripts (provided to it quoted in python code blocks), or sh shell scripts (provided to it quoted in sh code blocks).



In [ ]:

# The example code may download files from the internet, execute code, and interact with web pages. 
# Ensure you are in a safe environment before running the example code.

async def main() -> None:
    surfer = MultimodalWebSurfer(
        "WebSurfer",
        model_client=model_client,
    )

    file_surfer = FileSurfer( "FileSurfer",model_client=model_client)
    coder = MagenticOneCoderAgent("Coder",model_client=model_client)
    terminal = CodeExecutorAgent("ComputerTerminal",code_executor=LocalCommandLineCodeExecutor())
    
    team = MagenticOneGroupChat([surfer, file_surfer, coder, terminal], model_client=model_client)

    await Console(team.run_stream(task="What is the UV index in Melbourne today?"))



await main()


## This uses Magentic_one helper class with all the agents bundled together



In [ ]:



async def example_usage():
    client = model_client
    m1 = MagenticOne(client=client)
    task = "Write a Python script to fetch data from the OpenWeatherMap API, and save the file locally."
    result = await Console(m1.run_stream(task=task))
    print(result)


if __name__ == "__main__":
    await example_usage()
